<a href="https://colab.research.google.com/github/mattlsn/test/blob/main/Q_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# openCV 作業：
* 題目 ：請找一段約一分鐘左右的影片，利用上課時學過的影像處理功能，發揮所有創意和想像，將該影片加工特效，產生一段新影片，怕誤解同學的創意，請將你的亮點盡可能寫在影片中的右下角

* 上傳 ：導師會有告訴同學上傳空間, 或可上傳到Youtube 或自己的雲端空間, 把連結給老師(依導師規範為準)

* 繳交日期 ：課後二週

* 評分 ：亮點愈亮，創意愈多，分數愈高

* 參考 :<br>
https://www.youtube.com/watch?v=Fszu0L1JtqM

In [ ]:
import cv2

# 讀取影片檔案
cap = cv2.VideoCapture('case.mp4')

# 設置影片寬高、FPS、四字符代碼（編碼方式）
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('output_video.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (frame_width, frame_height))

# 獲取總幀數
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# 初始化 FastFeatureDetector
fast = cv2.FastFeatureDetector_create()

# 設定加入Sobel和Canny的起始和結束幀數
sobel_start_frame = 1800
sobel_end_frame = sobel_start_frame + 200
canny_start_frame = 2000
canny_end_frame = canny_start_frame + 200

# 開始處理每一幀
frame_count = 0
while True:
    ret, frame = cap.read()

    if ret:
        frame = cv2.resize(frame, (0, 0), fx=0.4, fy=0.4)

        if frame_count < total_frames:
            if frame_count < 200:
                # 前200幀：灰階播放
                display_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            elif frame_count < 400:
                # 接著的200幀：BGR播放
                display_frame = frame
            elif frame_count < 600:
                # 再接著的200幀：RGB播放
                display_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            elif frame_count < 800:
                # 再接著的200幀：R通道播放
                display_frame = frame.copy()
                display_frame[:, :, 1] = 0
                display_frame[:, :, 2] = 0
            elif frame_count < 1000:
                # 接著的200幀：G通道播放
                display_frame = frame.copy()
                display_frame[:, :, 0] = 0
                display_frame[:, :, 2] = 0
            elif frame_count < 1200:
                # 再接著的200幀：B通道播放
                display_frame = frame.copy()
                display_frame[:, :, 0] = 0
                display_frame[:, :, 1] = 0
            elif frame_count < 1400:
                # 再接著的200幀：水平翻轉（flip x）
                display_frame = cv2.flip(frame, 1)
            elif frame_count < 1600:
                # 再接著的200幀：垂直翻轉（flip y）
                display_frame = cv2.flip(frame, 0)
            elif frame_count < 1800:
                # 最後的200幀：同時水平垂直翻轉（flip xy）
                display_frame = cv2.flip(frame, -1)
            elif sobel_start_frame <= frame_count < sobel_end_frame:
                # Sobel效果
                display_frame = cv2.Sobel(frame, cv2.CV_64F, 1, 0, ksize=5)
                display_frame = cv2.convertScaleAbs(display_frame)
            elif canny_start_frame <= frame_count < canny_end_frame:
                # Canny效果
                display_frame = cv2.Canny(frame, 50, 150)
            else:
                # 其他情況：FastFeatureDetector
                keypoints = fast.detect(frame, None)
                display_frame = cv2.drawKeypoints(frame, keypoints, None, color=(255, 0, 0))

            # 在影片右下角註寫當前幀數
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(display_frame, 'Frame: {}'.format(frame_count), (10, 30), font, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # 顯示及保存影片
            cv2.imshow('Video', display_frame)
            out.write(display_frame)

        else:
            break  # 退出循環，播放完成
    else:
        break

    frame_count += 1

    # 按 'q' 退出
    if cv2.waitKey(1) == ord("q"):
        break

# 釋放 VideoWriter 對象
out.release()

# 釋放視訊捕獲對象和關閉所有窗口
cap.release()
cv2.destroyAllWindows()
